In [90]:
import numpy as np
import pandas as pd

In [52]:
def get_next(init, turn, b_in=1, b_out=-1, b_home=0.01):
    # If the household who's turn it is is empty,
    # nothing changes and on to the next household
    if init[turn] == "_":
        return [init, init, "empty house", turn, turn]

    # Figuring out what houses are available to move to
    blank_vec = [i for i, char in enumerate(init) if char == "_"]
    avail = [turn] + blank_vec

    # Set the utility to any non-available house to negative infinity
    u_avail = [-float('inf')] * len(init)
    for i in avail:
        u_avail[i] = 0
    u_avail[turn] = b_home

    # Computing the utility for all available houses
    for i in avail:
        # test is what the arrangement would be if moving (or staying)
        test = list(init)
        test[turn] = init[i]
        test[i] = init[turn]
        test = "".join(test)

        # Adding b_in if there is an ingroup person to the left...
        if i > 0 and init[turn] == test[i - 1]:
            u_avail[i] += b_in

        # ...or to the right
        if i < len(init) - 1 and init[turn] == test[i + 1]:
            u_avail[i] += b_in

        # Subtracting if there is an outgroup person to the left
        if i > 0 and (init[turn] != test[i - 1]) and (test[i - 1] not in ["_", ""]):
            u_avail[i] += b_out

        # Same for the right
        if i < len(init) - 1 and (init[turn] != test[i + 1]) and (test[i + 1] not in ["_", ""]):
            u_avail[i] += b_out

    # Figuring out where to move to.
    move = u_avail.index(max(u_avail))

    # Creating the new arrangement
    out = list(init)
    out[turn] = init[move]
    out[move] = init[turn]
    out = "".join(out)

    # Creating the output when there is a move
    if turn != move:
        return [init, out, f"the {init[turn]} in slot {turn} moves to {move}", turn, move]
    
    # Output when there is no move
    return [init, out, f"the {init[turn]} in slot {turn} stays", turn, move]

# Test the function
#initial_state = "AAABBAB___"
#turn = 7
#result = get_next(initial_state, turn)
#print(result)


In [53]:
#out_mat_test = [["initial", "ABABAB", "Start", 0, 1] for _ in range(100)]
#out_mat_test[0][3]

0

In [54]:
#0% 8

0

In [55]:
def run_schelling(init, b_in=1, b_out=-1, b_home=0.01, max_steps=100):
    out_mat = [["initial", init, "Start", 0, 1] for _ in range(max_steps)]

    for i in range(1, max_steps):
        last_turn = int(out_mat[i - 1][4])
        next_turn = 0 if out_mat[i - 1][3] != out_mat[i - 1][4] else (last_turn + 1) % len(init)

#        if next_turn == 0:
#            next_turn = len(init) - 1

        result = get_next(out_mat[i - 1][1], next_turn, b_in=b_in, b_out=b_out, b_home=b_home)
        out_mat[i] = result

        if i >= len(init):
            stays = sum(1 for row in out_mat[(i - len(init) + 1):(i + 1)] if row[3] == row[4])
            if stays == len(init):
                return out_mat[:i + 1]

    return out_mat


In [56]:
 #initial_state = "ABABABAB_"
# result = run_schelling(initial_state)
# for row in result:
#   print(row)

In [73]:
def display_schelling(init, b_in=1, b_out=0, b_home=0.01, max_steps=100, shorten=False):
    outmat = run_schelling(init=init, b_in=b_in, b_out=b_out, b_home=b_home, max_steps=max_steps)
    out_df = []

    for row in outmat:
        out_df.append([row[3], row[2], row[1]])

   # out_df = sorted(out_df, key=lambda x: x[0])

    if shorten:
        display_rows = []
        display_rows.append(out_df[0])
        for i in range(0, len(out_df)):
            if "moves to" in out_df[i][1]:
                display_rows.append(out_df[i])

        out_df = display_rows

    if not out_df:
        return pd.DataFrame(columns=["Turn", "Choice", "Outcome"])

    return pd.DataFrame(out_df, columns=["Turn", "Choice", "Outcome"])

In [74]:

 #Test the display_schelling function
#initial_state = "ABABABAB_"
#result = display_schelling(initial_state, shorten=True)

#result

,Turn,Choice,Outcome
0,0,Start,ABABABAB_
1,1,the B in slot 1 moves to 8,A_ABABABB
2,0,the A in slot 0 moves to 1,_AABABABB
3,4,the A in slot 4 moves to 0,AAAB_BABB
4,3,the B in slot 3 moves to 4,AAA_BBABB
5,6,the A in slot 6 moves to 3,AAAABB_BB
6,4,the B in slot 4 moves to 6,AAAA_BBBB


In [67]:
def end_schelling(init, b_in=1, b_out=0, b_home=0.01, max_steps=100):
    outmat = run_schelling(init=init, b_in=b_in, b_out=b_out, b_home=b_home, max_steps=max_steps)
    return outmat[-1][4]

# Test the end_schelling function
# initial_state = "AAABBAB___"
# result = end_schelling(initial_state)
# print(result)


9


In [102]:
def seg_meas(map):
    indiv_meas = [None] * len(map)
    
    for i in range(len(map)):
        self = map[i]
        nabe = []
        if i > 0:
            nabe.append(map[i - 1])
        if i < len(map) - 1:
            nabe.append(map[i + 1])
        
        same = nabe.count(self)
        diff = sum(1 for neighbor in nabe if neighbor != self and neighbor != "_")
        nonempty = sum(1 for neighbor in nabe if neighbor != "_")
        
        if self == "_":
            indiv_meas[i] = None
        else:
            indiv_meas[i] = (same - diff) / nonempty if nonempty != 0 else None
    
    return sum(filter(None, indiv_meas)) / (len(map) - indiv_meas.count(None))

# Test the seg_meas function
neighborhood_map = "AABBAABB"
result = seg_meas(neighborhood_map)
#print(result)
